## Day 11

https://adventofcode.com/2020/day/11

In [1]:
import copy

In [2]:
import aocd

In [3]:
test_data = """
L.LL.LL.LL
LLLLLLL.LL
L.L.L..L..
LLLL.LL.LL
L.LL.LL.LL
L.LLLLL.LL
..L.L.....
LLLLLLLLLL
L.LLLLLL.L
L.LLLLL.LL
"""

In [4]:
def split_data(raw):
    return [
        list(row.strip())
        for row in raw.splitlines()
        if row
    ]

In [5]:
# data = split_data(test_data)
data = split_data(aocd.get_data(day=11, year=2020))
len(data)

98

In [6]:
EMPTY = 'L'
OCCUPIED = '#'
FLOOR = '.'

In [7]:
class SeatMap:
    DIRECTIONS = (
        (-1, 0),  # left
        (1, 0),   # right
        (0, 1),   # up
        (0, -1),  # down
        (1, 1),   # upper-right
        (1, -1),  # lower-right
        (-1, 1),  # upper-left
        (-1, -1), # lower-left
    )
    
    def __init__(self, data, *, adjacency_rule):
        self.data = copy.deepcopy(data)
        self.adjacency_rule = adjacency_rule
        self.rows = len(data)

    @classmethod
    def copy(cls, seat_map):
        return cls(seat_map.data, adjacency_rule=seat_map.adjacency_rule)

    def cols(self, row) -> int:
        return len(self.data[row])
    
    def __eq__(self, other):
        if not isinstance(other, SeatMap):
            return False
        return self.data == other.data
    
    @property
    def seats(self):
        for row in range(self.rows):
            for col in range(self.cols(row)):
                yield row, col
    
    def is_valid_seat(self, row, col) -> bool:
        return (0 <= row < self.rows) and (0 <= col < self.cols(row))
    
    def current(self, row, col) -> str:
        return self.data[row][col]

    def change(self, row, col, value) -> None:
        self.data[row][col] = value

    def is_empty(self, row, col) -> bool:
        return self.current(row, col) == EMPTY
    
    def is_occupied(self, row, col) -> bool:
        return self.current(row, col) == OCCUPIED
    
    def is_floor(self, row, col) -> bool:
        return self.current(row, col) == FLOOR
    
    @staticmethod
    def new_seat(row, col, direction) -> tuple:
        dx, dy = direction
        return row + dy, col + dx
    
    def find_adjacent_seat(self, row, col, direction, *, verbose=False):
        return self.adjacency_rule(self, row, col, direction, verbose=verbose)

    def adjacent_seats(self, row, col, *, verbose=False):
        for d in self.DIRECTIONS:
            seat = self.find_adjacent_seat(row, col, d, verbose=verbose)
            if seat:
                yield seat
    
    def count(self, value) -> int:
        return sum(row.count(value) for row in self.data)
    
    def count_occupied_adjacent(self, row, col, *, verbose=False):
        return sum(
            self.is_occupied(x, y)
            for x, y in self.adjacent_seats(row, col, verbose=verbose)
        )

In [8]:
def seat_change(seat_map, row, col, *, threshold) -> str:
    current = seat_map.current(row, col)
    occupied = seat_map.is_occupied(row, col)
    empty = seat_map.is_empty(row, col)
    occupied_adjacent = seat_map.count_occupied_adjacent(row, col)
    if empty and occupied_adjacent == 0:
        return OCCUPIED
    if occupied and occupied_adjacent >= threshold:
        return EMPTY
    return current

In [9]:
def update(seat_map, *, threshold) -> SeatMap:
    new_seat_map = SeatMap.copy(seat_map)  # makes a deep copy
    for row, col in new_seat_map.seats:
        # note: simultaneous changes, so use "old" map held constant
        new_value = seat_change(seat_map, row, col, threshold=threshold)
        new_seat_map.change(row, col, new_value)
    return new_seat_map

In [10]:
class IterationLimitError(RuntimeError):
    def __init__(self, msg, progress):
        super().__init__(msg)
        self.progress = progress

In [11]:
class ProgressSaver:
    def __init__(self):
        self.progress = []
        
    def __repr__(self):
        return repr(self.progress)
        
    def append(self, count: int, old_map: SeatMap, new_map: SeatMap) -> None:
        empty = old_map.count(EMPTY)
        occ = old_map.count(OCCUPIED)
        new_empty = new_map.count(EMPTY)
        new_occ = new_map.count(OCCUPIED)
        self.progress.append((count, (empty, occ), (new_empty, new_occ)))

In [12]:
def iterate(seat_map, *, threshold, limit):
    seat_map = SeatMap.copy(seat_map)
    count = 0
    progress = ProgressSaver()
    while count < limit:
        count += 1
        new_seat_map = update(seat_map, threshold=threshold)
        progress.append(count, seat_map, new_seat_map)
        if new_seat_map == seat_map:
            return new_seat_map
        seat_map = SeatMap.copy(new_seat_map)
    raise IterationLimitError('hit limit', progress)

### Solution to Part 1

In [13]:
def adjacent_seat_rule(seat_map, row, col, direction, *, verbose=False):
    row, col = seat_map.new_seat(row, col, direction)
    if seat_map.is_valid_seat(row, col) and not seat_map.is_floor(row, col):
        return row, col
    return None

In [14]:
def run(data, *, adjacency_rule, threshold=4, limit=200):
    seat_map = SeatMap(data, adjacency_rule=adjacency_rule)
    try:
        new_seat_map = iterate(seat_map, threshold=threshold, limit=limit)
    except RuntimeError as exc:
        progress = exc.progress
        raise
    return new_seat_map.count(OCCUPIED)

In [15]:
run(data, adjacency_rule=adjacent_seat_rule)

2299

### Solution to Part 2

In [16]:
def part2_adjacent_seat_rule(seat_map, row, col, direction, *, verbose=False):
    row, col = seat_map.new_seat(row, col, direction)
    while seat_map.is_valid_seat(row, col):
        if not seat_map.is_floor(row, col):
            return row, col
        row, col = seat_map.new_seat(row, col, direction)
    return None

In [17]:
run(data, adjacency_rule=part2_adjacent_seat_rule, threshold=5, limit=300)

2047